## Preprocessing

In [1]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 43.8 MB/s 


In [2]:
# Import our dependencies
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import files
#  Import and read the charity_data.csv.
import pandas as pd
import keras_tuner as kt 
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [3]:
import io
 
application_df = pd.read_csv(io.StringIO(uploaded['charity_data.csv'].decode('utf-8')))
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
filtered_df = application_df.drop(columns=["NAME", "EIN"])
filtered_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
filtered_cat = filtered_df.dtypes[filtered_df.dtypes == "object"].index.tolist()
filtered_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
filtered_df[filtered_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
app_type = filtered_df['APPLICATION_TYPE'].value_counts().reset_index()
app_type

,index,APPLICATION_TYPE
0,T3,27037
1,T4,1542
2,T6,1216
3,T5,1173
4,T19,1065
5,T8,737
6,T7,725
7,T10,528
8,T9,156
9,T13,66


In [8]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = []
application_types_to_replace = app_type[app_type['APPLICATION_TYPE'] <= 156]['index']

# Replace in dataframe
for app in application_types_to_replace:
    filtered_df['APPLICATION_TYPE'] = filtered_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
filtered_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
class_count = filtered_df['CLASSIFICATION'].value_counts().reset_index()
class_count

,index,CLASSIFICATION
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
...,...,...
66,C4120,1
67,C8210,1
68,C2561,1
69,C4500,1


In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_filtered = []
class_count_filtered = class_count[class_count['CLASSIFICATION'] > 1]
class_count_filtered

,index,CLASSIFICATION
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
5,C7000,777
6,C1700,287
7,C4000,194
8,C5000,116
9,C1270,114


In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = []
classifications_to_replace = class_count[class_count['CLASSIFICATION'] < 1883]['index']

# Replace in dataframe
for cls in classifications_to_replace:
    filtered_df['CLASSIFICATION'] = filtered_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
filtered_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
filtered_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
numerical_df = pd.get_dummies(filtered_df)
numerical_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
# Split our preprocessed data into our features and target arrays
y = numerical_df["IS_SUCCESSFUL"].values
X = numerical_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

<ipython-input-14-775795f8e043>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = numerical_df.drop(["IS_SUCCESSFUL"],1).values


In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=5), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [17]:
# Import the kerastuner library

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [18]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 23s]
val_accuracy: 0.7313119769096375

Best val_accuracy So Far: 0.7337609529495239
Total elapsed time: 00h 25m 38s


In [19]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 11, 'num_layers': 3, 'units_0': 56, 'units_1': 71, 'units_2': 11, 'units_3': 96, 'units_4': 31, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 21, 'num_layers': 4, 'units_0': 81, 'units_1': 61, 'units_2': 81, 'units_3': 41, 'units_4': 116, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 16, 'num_layers': 1, 'units_0': 66, 'units_1': 116, 'units_2': 6, 'units_3': 6, 'units_4': 96, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}


In [20]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5548 - accuracy: 0.7338 - 555ms/epoch - 2ms/step
Loss: 0.5547618269920349, Accuracy: 0.7337609529495239
268/268 - 1s - loss: 0.5531 - accuracy: 0.7336 - 586ms/epoch - 2ms/step
Loss: 0.5531494617462158, Accuracy: 0.7336443066596985
268/268 - 1s - loss: 0.5533 - accuracy: 0.7332 - 541ms/epoch - 2ms/step
Loss: 0.5532716512680054, Accuracy: 0.7331778407096863


In [21]:
#In the above exercise, optimisation was performed by varying the number of layers between 1 and 5, varying the number of input parameters between 1 and 120, varying the number of neurons in the hidden layers with the number of neurons varying between 1 and 120. The activation functions have been tested for relu and tanh.

In [24]:
# Creating model based on the top hyperparameter

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units = 11, activation = 'relu', input_dim = 43))
# First hidden layer
nn.add(tf.keras.layers.Dense(units = 56, activation = 'relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 71, activation = 'relu'))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 11, activation = 'relu'))

nn.add(tf.keras.layers.Dense(units = 96, activation = 'relu'))

nn.add(tf.keras.layers.Dense(units = 31, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs =20)

Epoch 1/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5783 - accuracy: 0.7120
Epoch 2/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5595 - accuracy: 0.7253
Epoch 3/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5566 - accuracy: 0.7269
Epoch 4/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5538 - accuracy: 0.7277
Epoch 5/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5523 - accuracy: 0.7280
Epoch 6/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7289
Epoch 7/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5507 - accuracy: 0.7301
Epoch 8/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5494 - accuracy: 0.7304
Epoch 9/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5494 - accuracy: 0.7313
Epoch 10/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5484 - accuracy: 0.7311

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5561 - accuracy: 0.7277 - 551ms/epoch - 2ms/step
Loss: 0.556114673614502, Accuracy: 0.7276967763900757


In [27]:
# Exporting the model to HDF5 file
nn.save('AlphabetSoupCharity_Optimisation.h5')
files.download('AlphabetSoupCharity_Optimisation.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>